### Comparison against author's dataset

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
import pandas as pd
readpath = '../src/data/bank.csv'
df = pd.read_csv(readpath)
df = df.loc[:, df.columns != 'Unnamed: 0']
print(df.head())
print(f"Number of columns: {len(df.columns)}")

   age  Experience  Income  Family     CCAvg  Education  Mortgage  \
0   52          15     140       2  7.737671          2        32   
1   51           0     150       3  5.073039          1       328   
2   32          18     144       3  6.295617          3        59   
3   51          20     159       1  5.495390          3       275   
4   38          17     154       2  3.225856          3       189   

   SecuritiesAccount  CDAccount  Online  CreditCard  Personal Loan  
0                  1          1       1           0              1  
1                  1          1       0           1              1  
2                  0          1       0           1              1  
3                  0          1       1           0              1  
4                  1          1       1           1              1  
Number of columns: 12


In [3]:
from sklearn.model_selection import train_test_split

X = df.loc[:, df.columns != 'Personal Loan']
y = df['Personal Loan']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [4]:
from src.data.load_data import preprocess_data
from src.data.load_data import preprocess_num

num_cols = ['age','Experience','Income','Family','CCAvg','Education','Mortgage']

X_train_raw,X_test_raw = X_train,X_test

X_train_num, X_test_num, datasetdf_num = preprocess_num(X_train, X_test, df, num_cols)
X_train, X_test, datasetdf = preprocess_data(X_train, X_test, df, num_cols)
print(X_train_num.head())

           age  Experience    Income    Family     CCAvg  Education  Mortgage  \
3865  0.984124    1.061265 -0.396133  0.487610 -0.135698  -1.344684 -0.780557   
5753 -0.294585   -0.374082 -0.663640  0.487610 -1.123458   1.216779  1.321462   
379  -0.111912   -1.360883  0.620392  1.411979  1.266926  -0.063953  0.689214   
2935 -0.385921   -0.463791  1.012735 -1.361130  1.152487   1.216779 -0.780557   
6186 -0.477258   -0.912337 -1.483993 -1.361130 -1.009661   1.216779  0.171920   

      SecuritiesAccount  CDAccount  Online  CreditCard  
3865                  0          0       1           0  
5753                  0          0       1           0  
379                   0          1       1           0  
2935                  0          1       1           0  
6186                  0          0       0           0  


In [5]:
from src.model.train_model import train_classifier_LR

clf = train_classifier_LR(X_train, y_train, random_state=42)
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.9633507853403142


In [6]:
from src.ontology.generate_rules import *

X_train_positive = X_train_num[y_train == 1]
X_train_negative = X_train_num[y_train == 0]
positive_profile = summarize_features(X_train_positive)
positive_profile
negative_profile = summarize_features(X_train_negative)
negative_profile
#p_statistical = extract_statistical_rules(X_train_positive, X_train_raw, num_quantiles=(0.1, 0.9), cat_threshold=0.85)
p_statistical = extract_statistical_rules(X_train_positive, X_train_num, num_quantiles=(0.1, 0.9), cat_threshold=0.85)
p_statistical
p_permutation_imp_top_features = top_features(clf, X_train, y_train, n_repeats=5, random_state=42)
p_permutation_imp_top_features
p = combined_features(p_permutation_imp_top_features, p_statistical)
p_num = {key: value for key, value in p.items() if isinstance(value, tuple)}
p_cat = {key: value for key, value in p.items() if isinstance(value, str)}
print(p_num)
print(p_cat)

{'age': (-1.299, 1.258), 'Experience': (-1.361, 1.33), 'Income': (0.424, 1.53), 'Family': (-1.361, 1.412), 'CCAvg': (0.053, 1.835), 'Education': (-0.064, 1.217), 'Mortgage': (-0.781, 2.069)}
{'CDAccount': '1'}


In [7]:
from src.utilities.desired_space import desired_space
X_test_negative = X_test[y_test==0]
x= X_test_negative[0:1] #test instance #select first row as dataFrame
p_num = p_num
p_cat = p_cat
space = datasetdf[datasetdf['Personal Loan'] == 1] #filter instances if they have the desired y
desired_space = space.drop(['Personal Loan'], axis=1) #drop y
#desired_space = desired_space(1,datasetdf) #did preprocessing also to datasetdf
#cat_f= X_train.select_dtypes(include=['bool']) #categorical features
cat_f = ['SecuritiesAccount', 'CDAccount', 'Online', 'CreditCard']
num_f = ['age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education', 'Mortgage']
#num_f= X_train.select_dtypes(include=['int64', 'float64']) #numerical features
protect_f= [] # Personal status and sex (X_train.head() shows only 3 values (columns) while there are 5) #protected features
features= X_train.columns #all features
t= 1 #desired outcome
f= clf #black-box model for which we want to find CF
X= X_train #training data
step = {attribute:0.01 for attribute in p_num.keys()} #was 0.01

f2change = p.keys()

In [8]:
f2change

dict_keys(['age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education', 'Mortgage', 'CDAccount'])

In [9]:
from src.utilities.UFCE import SF
#cf = SF(x,X,cat_f,p,f,t,step)
cf = SF(x,X_train,p_num,p_cat,f,t,step) # changed from X_train_raw to X_train
print(cf)

[           age  Experience  Income   Family     CCAvg  Education  Mortgage  \
3478 -1.025276   -0.912337  1.2585  0.48761 -0.440675  -1.344684 -0.780557   

      SecuritiesAccount  CDAccount  Online  CreditCard  
3478                  0          0       0           0  ,            age  Experience    Income   Family     CCAvg  Education  Mortgage  \
3478 -1.025276   -0.912337  1.223313  0.48761 -0.440675  -1.344684 -0.780557   

      SecuritiesAccount  CDAccount  Online  CreditCard  
3478                  0          0       0           0  ]


In [10]:
print(clf.predict(cf[0]))

[1]


In [11]:
from itertools import combinations
from src.utilities.mutual_information import mi_score
from src.utilities.mutual_information import MI

feature_pairs = list(combinations(list(features), 2))
print(feature_pairs[:5])
mi_list = MI(feature_pairs,
             X_train)  #used all features available #original authors do over entire X but it is not preprocessed... Which i think is nessecary? #also works for bool one-hot encoding 
# top 5 
print(mi_list[:5])
### Nearest Neighbour: KD Tree
from src.utilities.nearest_neighbours import FNN

nn = FNN(desired_space, x,
         3.5)  #radius arbitrarily set; can experiment with how close we want it to be to x #3.5 for euclidean distance is around 0.5 for each of the 48 features difference from x, get 3 datapoints
print(nn)
### Subspace: Intervals
from src.utilities.nearest_neighbours import intervals
from src.utilities.nearest_neighbours import intervals2

#subspace = intervals(nn,p,f2change,x) 
subspace = intervals2(nn, p_num, p_cat, f2change, x)
print(subspace)  #correct output

[('age', 'Experience'), ('age', 'Income'), ('age', 'Family'), ('age', 'CCAvg'), ('age', 'Education')]
[('age', 'Experience'), ('Income', 'CCAvg'), ('CCAvg', 'CDAccount'), ('Income', 'CDAccount'), ('CCAvg', 'Mortgage')]
           age  Experience    Income    Family     CCAvg  Education  Mortgage  \
3762 -1.207948   -1.091755  0.031878 -0.436760  0.333147  -1.344684 -0.780557   
5587 -1.755966   -1.630010  0.531223 -0.436760 -0.258599   1.216779 -0.780557   
4904 -1.664630   -1.540301  0.049712 -1.361130  0.014514  -0.063953 -0.780557   
3658 -1.664630   -1.540301  0.281551 -1.361130  0.151071  -0.063953 -0.780557   
7577 -1.573294   -1.450592  0.174548 -0.436760 -0.622750  -0.063953 -0.780557   
...        ...         ...       ...       ...       ...        ...       ...   
1274 -0.385921    0.881846  0.477722  1.411979  0.932044  -0.063953  0.401829   
1586 -1.025276   -1.271173  0.745229 -0.436760  0.832978  -0.063953  1.305040   
1822 -0.842603   -0.284373  0.620392  1.411979  0.81

In [12]:
from src.utilities.UFCE import DF

CFs = DF(datasetdf, x, X_train, subspace, mi_list, cat_f, num_f, features, protect_f, f, t)
# print(CFs)
# print(len(CFs))

In [13]:
import numpy as np

bad_CFs = []

for i, cf in enumerate(CFs):
    prediction = f.predict(cf)

    # If the model returns an array, extract the scalar
    if isinstance(prediction, (np.ndarray, list)):
        prediction = prediction[0]

    if prediction != t:
        print(f"CF #{i} has incorrect prediction: {prediction} (expected: {t})")
        bad_CFs.append((i, prediction))
        
if not bad_CFs:
    print("All good: All CFs have prediction == t")

All good: All CFs have prediction == t


In [14]:
print(x)
print(CFs[0])
print(t)
print(f.predict(CFs[0]))

           age  Experience    Income   Family     CCAvg  Education  Mortgage  \
3478 -1.025276   -0.912337 -0.859811  0.48761 -0.440675  -1.344684 -0.780557   

      SecuritiesAccount  CDAccount  Online  CreditCard  
3478                  0          0       0           0  
           age  Experience  Income   Family     CCAvg  Education  Mortgage  \
3478 -1.025276   -0.912337    1.21  0.48761  0.618328  -1.344684 -0.780557   

      SecuritiesAccount  CDAccount  Online  CreditCard  
3478                  0          0       0           0  
1
[1]


In [15]:
print("original: ")
print(x)
print("counterfactual: ")
print(CFs[0])
print("Differences between original and counterfactual:")

# Compare x and z for all columns
differences = (x != CFs[0]).iloc[0]

# Print only the differing columns with their values in x and z
for col in x.columns[differences]:
    print(f"{col}: original = {x.iloc[0][col]}, counterfactual = {CFs[0].iloc[0][col]}")

original: 
           age  Experience    Income   Family     CCAvg  Education  Mortgage  \
3478 -1.025276   -0.912337 -0.859811  0.48761 -0.440675  -1.344684 -0.780557   

      SecuritiesAccount  CDAccount  Online  CreditCard  
3478                  0          0       0           0  
counterfactual: 
           age  Experience  Income   Family     CCAvg  Education  Mortgage  \
3478 -1.025276   -0.912337    1.21  0.48761  0.618328  -1.344684 -0.780557   

      SecuritiesAccount  CDAccount  Online  CreditCard  
3478                  0          0       0           0  
Differences between original and counterfactual:
Income: original = -0.859811069298052, counterfactual = 1.21
CCAvg: original = -0.440674747455996, counterfactual = 0.6183278542265571


In [16]:
from src.evaluation.metrics import *

In [17]:
X_test_negative_subset = X_test_negative[0:50]

In [18]:
actionable_features = f2change
feasibility_score = proximity_score_num = proximity_score_cat = sparsity_score = 0.0
cf_counter_SF = 0
for idx, row in X_test_negative_subset.iterrows():
    x = row.to_frame().T
    nn = FNN(desired_space,x,3.5)
    subspace = intervals2(nn,p_num,p_cat,f2change,x)
    cf_SF = SF(x,X_train,p_num,p_cat,f,t,step)
    if(len(cf_SF)!=0):
        cf = cf_SF[0] if isinstance(cf_SF, list) else cf_SF
        compute_feasibility(x, cf, actionable_features, clf, 1, X_train,0.5)
        if compute_feasibility(x, cf, actionable_features, clf, 1, X_train,0.5):
            feasibility_score = feasibility_score + 1
        
        sparsity_score = sparsity_score + compute_sparsity(x, cf)
        cat_cols = list(x.columns.difference(num_cols))
        prox_num,prox_cat = compute_proximity(x, cf, num_cols,cat_cols)
        proximity_score_num = proximity_score_num + prox_num
        proximity_score_cat = proximity_score_cat + prox_cat
        print("Evaluation for SF done for: ",idx)
        cf_counter_SF += 1
    else:
        print("Empty CF for: ",idx)    

Evaluation for SF done for:  3478
Empty CF for:  5024
Evaluation for SF done for:  6621
Empty CF for:  5800
Evaluation for SF done for:  3614
Evaluation for SF done for:  4545
Evaluation for SF done for:  6215
Evaluation for SF done for:  6819
Evaluation for SF done for:  6676
Evaluation for SF done for:  2776
Evaluation for SF done for:  7474
Evaluation for SF done for:  5700
Evaluation for SF done for:  6329
Evaluation for SF done for:  3215
Evaluation for SF done for:  3836
Evaluation for SF done for:  7427
Empty CF for:  3934
Evaluation for SF done for:  7286
Evaluation for SF done for:  5107
Evaluation for SF done for:  6436
Empty CF for:  3920
Evaluation for SF done for:  5271
Evaluation for SF done for:  3609
Evaluation for SF done for:  6674
Evaluation for SF done for:  4187
Evaluation for SF done for:  7537
Evaluation for SF done for:  4746
Evaluation for SF done for:  2882
Empty CF for:  4063
Evaluation for SF done for:  5178
Evaluation for SF done for:  3191
Evaluation for S

In [19]:
print(feasibility_score)
print(proximity_score_num)
print(proximity_score_cat)
print(sparsity_score)

31.0
62.63074944250488
28.0
2.9999999999999987


In [20]:
avg_feasibility = feasibility_score / cf_counter_SF
print(avg_feasibility)

0.8611111111111112


In [21]:
avg_proximity_num = proximity_score_num / cf_counter_SF
print(avg_proximity_num)

1.73974304006958


In [22]:
avg_proximity_cat = proximity_score_cat / cf_counter_SF
print(avg_proximity_cat)

0.7777777777777778


In [23]:
avg_sparsity = sparsity_score / cf_counter_SF
print(avg_sparsity)

0.0833333333333333


In [24]:
actionable_features = f2change
feasibility_score_DF = proximity_score_num_DF = proximity_score_cat_DF = sparsity_score_DF = 0.0
cf_counter_DF = 0
for idx, row in X_test_negative_subset.iterrows():
    x = row.to_frame().T
    nn = FNN(desired_space,x,3.5)
    subspace = intervals2(nn,p_num,p_cat,f2change,x)

    cf_DF = DF(datasetdf,x,X_train,subspace,mi_list,cat_f,num_f,features,protect_f,f,t)
    unique_cf_signatures = set()
    unique_counterfactuals = []
    for cf in cf_DF:
        # Align columns
        common_cols = x.columns.intersection(cf.columns)
        x_common = x[common_cols]
        cf_common = cf[common_cols]
        
        # Find differences
        differences = (x_common != cf_common).iloc[0]
        
        # Represent difference as a tuple of (feature, original_val, cf_val)
        diff_signature = tuple(
            (col, x[col].values[0], cf[col].values[0])
            for col in common_cols[differences]
        )
        
        # Add only if this signature is new
        if diff_signature not in unique_cf_signatures:
            unique_cf_signatures.add(diff_signature)
            unique_counterfactuals.append(cf)
    best_cf = None
    min_total_proximity = float('inf')
    
    for cf in unique_counterfactuals:
        if len(cf) != 0:
            # Compute proximity for current CF
            cat_cols = list(x.columns.difference(num_cols))
            prox_num, prox_cat = compute_proximity(x, cf, num_cols, cat_cols)
            total_proximity = prox_num + prox_cat
    
            # Keep the CF with the lowest proximity
            if total_proximity < min_total_proximity:
                min_total_proximity = total_proximity
                best_cf = cf
        else:
            print("Empty CF for: ", idx)
    
    # Evaluate only the best CF
    if best_cf is not None:
        print("Best CF selected for: ", idx)
  
        # Feasibility
        if compute_feasibility(x, best_cf, actionable_features, clf, 1, X_train, 0.5):
            feasibility_score_DF += 1
    
        # Sparsity
        sparsity_score_DF += compute_sparsity(x, best_cf)
    
        # Proximity
        prox_num, prox_cat = compute_proximity(x, best_cf, num_cols, cat_cols)
        proximity_score_num_DF += prox_num
        proximity_score_cat_DF += prox_cat
    
        cf_counter_DF += 1
    else:
        print("No valid CF found for: ", idx)
    # for cf in unique_counterfactuals:
    #     if(len(cf)!=0):
    #         compute_feasibility(x, cf, actionable_features, clf, 1, X_train,0.5)
    #         if compute_feasibility(x, cf, actionable_features, clf, 1, X_train,0.5):
    #             feasibility_score_DF = feasibility_score_DF + 1
            
    #         sparsity_score_DF = sparsity_score_DF + compute_sparsity(x, cf)
    #         cat_cols = list(x.columns.difference(num_cols))
    #         prox_num,prox_cat = compute_proximity(x, cf, num_cols,cat_cols)
    #         proximity_score_num_DF = proximity_score_num_DF + prox_num
    #         proximity_score_cat_DF = proximity_score_cat_DF + prox_cat
    #         print("Evaluation for DF done for: ",idx)
    #         cf_counter_DF += 1
    #     else:
    #         print("Empty CF for: ",idx)

Best CF selected for:  3478
Best CF selected for:  5024
Best CF selected for:  6621
No valid CF found for:  5800
Best CF selected for:  3614
Best CF selected for:  4545
Best CF selected for:  6215
Best CF selected for:  6819
Best CF selected for:  6676
Best CF selected for:  2776
Best CF selected for:  7474
Best CF selected for:  5700
Best CF selected for:  6329
Best CF selected for:  3215
Best CF selected for:  3836
Best CF selected for:  7427
Best CF selected for:  3934
Best CF selected for:  7286
Best CF selected for:  5107
Best CF selected for:  6436
No valid CF found for:  3920
Best CF selected for:  5271
Best CF selected for:  3609
Best CF selected for:  6674
Best CF selected for:  4187
Best CF selected for:  7537
Best CF selected for:  4746
Best CF selected for:  2882
Best CF selected for:  4063
Best CF selected for:  5178
Best CF selected for:  3191
Best CF selected for:  6463
No valid CF found for:  3482
No valid CF found for:  5684
Best CF selected for:  4955
No valid CF foun

In [25]:
print(feasibility_score_DF)
print(proximity_score_num_DF)
print(proximity_score_cat_DF)
print(sparsity_score_DF)

41.0
72.7385203900729
24.666666666666668
6.727272727272723


In [26]:
avg_feasibilit_DF = feasibility_score_DF / cf_counter_DF
print(avg_feasibilit_DF)

0.9761904761904762


In [27]:
avg_proximity_num_DF = proximity_score_num_DF / cf_counter_DF
print(avg_proximity_num_DF)

1.731869533096974


In [28]:
avg_proximity_cat_DF = proximity_score_cat_DF / cf_counter_DF
print(avg_proximity_cat_DF)

0.5873015873015873


In [29]:
avg_sparsity_DF = sparsity_score_DF / cf_counter_DF
print(avg_sparsity_DF)

0.16017316017316008


In [30]:
cf_feasibility = {}
cf_sparsity = {}
cf_proximity_num = {}
cf_proximity_cat = {}

x = X_test_negative[5:6]
nn = FNN(desired_space,x,3.5)
subspace = intervals2(nn,p_num,p_cat,f2change,x)
cf_DF = DF(datasetdf,x,X_train,subspace,mi_list,cat_f,num_f,features,protect_f,f,t)
unique_cf_signatures = set()
unique_counterfactuals = []
for cf in cf_DF:
    # Align columns
    common_cols = x.columns.intersection(cf.columns)
    x_common = x[common_cols]
    cf_common = cf[common_cols]
    
    # Find differences
    differences = (x_common != cf_common).iloc[0]
    
    # Represent difference as a tuple of (feature, original_val, cf_val)
    diff_signature = tuple(
        (col, x[col].values[0], cf[col].values[0])
        for col in common_cols[differences]
    )
    
    # Add only if this signature is new
    if diff_signature not in unique_cf_signatures:
        unique_cf_signatures.add(diff_signature)
        unique_counterfactuals.append(cf)
for idx, cf in enumerate(unique_counterfactuals):
    if(len(cf)!=0):
        compute_feasibility(x, cf, actionable_features, clf, 1, X_train,0.5)
        if compute_feasibility(x, cf, actionable_features, clf, 1, X_train,0.5):
            cf_feasibility[idx] = 1
        else:
            cf_feasibility[idx] = 0
        cf_sparsity[idx] = compute_sparsity(x, cf)
        cat_cols = list(x.columns.difference(num_cols))
        prox_num,prox_cat = compute_proximity(x, cf, num_cols,cat_cols)
        cf_proximity_num[idx] = prox_num
        cf_proximity_cat[idx] = prox_cat

In [31]:
cf_feasibility

{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1}

In [32]:
cf_sparsity

{0: 0.18181818181818182,
 1: 0.18181818181818182,
 2: 0.18181818181818182,
 3: 0.18181818181818182,
 4: 0.18181818181818182,
 5: 0.18181818181818182}

In [33]:
cf_proximity_num

{0: 1.3210830539118215,
 1: 2.05723147491461,
 2: 0.9916235993344971,
 3: 1.4587172176439454,
 4: 1.1922213419863141,
 5: 1.483550076867324}

In [34]:
cf_proximity_cat

{0: 1.0, 1: 0.5, 2: 0.5, 3: 1.0, 4: 1.0, 5: 1.0}